# Grama Model Evaluation

*Purpose*: Once we've done the hard work of building a grama model, we can use a variety of tools to use the model. Evaluations are core to using grama.

### TODO 


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

To focus this exercise on using models, let's load up a model to play with:


In [ ]:
from grama.models import make_plate_buckle
md_plate = make_plate_buckle()

# Evaluation

Recall that there are four classes of verb in grama; *evaluation* verbs take a model as an input and produce data as an output. Once we've generated that data, we can use visualization and other data tools to learn something about the model.



![Grama verb class diagram](./images/verb-classes.png)


# Manual Input Values

There are two core evaluations---`ev_df()` and `tf_md()`---that require us to specify all the input values. These are manual but fundamental tools for working with a model.

## DataFrame evaluation `ev_df()`

As the prefix implies, `ev_df()` takes a model as an input and returns data as an output. Since we need to specify all model input values, it is most convenient to pair `ev_df()` with the DataFrame constructor `df_make()`. Let's talk about some practical considerations when putting these tools together.


### __qX__ Set the input values


In [ ]:
# TASK: Complete the following code
(
    md_plate

)

## Evaluation as transformation `tf_md()`

### TODO


In [ ]:
from grama.data import df_stang
df_stang.head(6)

### __qX__ Use a model as a transform


In [ ]:
(
    df_stang
    >> gr.tf_mutate(
        m=1,
        L=0.001,
        t=0.06,
        h=12,
        w=12,
    )

)

# Mixed Manual & Automatic

## Nominal Evaluation


### __qX__ Evaluate at nominal values


In [ ]:
(
    md_plate

)

*Observations*

- ...
  - (Your response here)


### __qX__ Sweep key variables


In [ ]:
(
    md_plate
    >> gr.ev_nominal(
        df_det=gr.df_grid(
            w=12,
            t=1/32,
            L=0.0016,

        )
    )
    
    >> gr.ggplot(gr.aes("h", "g_buckle"))
    + gr.geom_line(gr.aes(color="factor(m)"))
)

*Observations*

- ...
  - (Your response here)


## Random Sampling


### __qX__ Evaluate a simple random sample




In [ ]:
(
    md_plate

)

*Observations*

- ...
  - (Your response here)


### __qX__ Skip evaluation


In [ ]:
(
    md_plate
    >> gr.ev_sample(
        n=1e2, 
        df_det="nom",

    )
    
    # NOTE: No need to edit
    >> gr.pt_auto()
)

*Observations*

- ...
  - (Your response here)


### __qX__ Sweeps with sampling


In [ ]:
(
    md_plate
    # Uncomment, replace ev_nominal() with ev_sample(),
    # adjust the arguments to ev_sample()
    # >> gr.ev_nominal(
    #     df_det=gr.df_grid(
    #         w=12,
    #         t=1/32,
    #         L=0.0016,
    #         m=[1, 2, 3],
    #         h=gr.linspace(6, 12, 10)
    #     )
    # )

    
    ## NOTE: No need to edit below; use to inspect results
    # Compute low, middle, high values at each m, h
    >> gr.tf_group_by(DF.m, DF.h)
    >> gr.tf_summarize(
        g_lo=gr.quant(DF.g_buckle, p=0.25),
        g_mu=gr.median(DF.g_buckle),
        g_hi=gr.quant(DF.g_buckle, p=0.75),
    )
    # Visualize
    >> gr.ggplot(gr.aes("h", "g_mu", color="factor(m)"))
    + gr.geom_ribbon(gr.aes(ymin="g_lo", ymax="g_hi"), fill=None)
    + gr.geom_line()
)

*Observations*

- ...
  - (Your response here)


# Automatic Input Values

## Contour Evaluation


### __qX__ Visualize function contours


In [ ]:
# TASK: Complete the contour evaluation arguments
(
    # NOTE: No need to edit this model
    gr.Model("Contour Demo")
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            f=df.x**2 + df.y**2,
            g=df.x + df.y,
        ),
        var=["x", "y"],
        out=["f", "g"],
    )
    >> gr.cp_bounds(
        x=(-1, +1),
        y=(-1, +1),
    )
    
    # TASK: Complete the arguments for ev_contour()
    >> gr.ev_contour(
        # HINT: Use Shift+Tab to consult the documentation

    )
    
    # NOTE: No need to edit; use to visualize
    >> gr.pt_auto()
)

*Observations*

- ...
  - (Your response here)


### __qX__ Find specific contour levels

Use `gr.ev_contour()` to find the contour of `g` that is tangent to the contour where `f == 1`.


In [ ]:
# TASK: Find the tangent contour of `g`
df_contour = (
    # NOTE: No need to edit this model
    gr.Model("Contour Demo")
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            f=df.x**2 + df.y**2,
            g=df.x + df.y,
        ),
        var=["x", "y"],
        out=["f", "g"],
    )
    >> gr.cp_bounds(
        x=(0, +1),
        y=(0, +1),
    )
    
    # TASK: Use specific levels to find the tangent
    >> gr.ev_contour(
        var=["x", "y"],
        out=["f", "g"],
        # HINT: Use Shift+Tab to consult the documentation

    )
)

# NOTE: No need to edit; use to visualize
gr.plot_auto(df_contour)

## Constrained Minimization


In [ ]:
# NOTE: No need to edit
df_opt = (
    # Set up a model
    gr.Model("Contour Demo")
    >> gr.cp_vec_function(
        fun=lambda df: gr.df_make(
            f=df.x**2 + df.y**2,
            g=df.x + df.y - 1.42,
        ),
        var=["x", "y"],
        out=["f", "g"],
    )
    >> gr.cp_bounds(
        x=(0, +1),
        y=(0, +1),
    )
    
    # Minimize the objective with a constraint
    >> gr.ev_min(
        out_min="f",
        out_geq=["g"],
    )
)

df_opt

In [ ]:
# NOTE: No need to edit; this visualizes the optimization results
# in the context of the contour data
(
    df_contour
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_segment(
        gr.aes(
            xend="x_end", 
            yend="y_end",
            color="level",
            linetype="out",
          )
    )
    + gr.geom_point(data=df_opt, size=4)
)

# List of evaluation routines

| Verb | Input values | Description |
|------|-------|-------------|
| `ev_df()` | Manual | DataFrame evaluation |
| `tf_md()` | Manual | Model as transformation |
| `ev_nominal()` | Mixed | Nominal values for random inputs |
| `ev_sample()` | Mixed | Random values for random inputs |
| `ev_conservative()` | Mixed | Conservative values for random inputs |
| `ev_contour()` | Auto | Generate contour plot data |
| `ev_sinews()` | Auto | Generate sinew plot data |
| `ev_min()` | Auto | Constrained minimization |
